<a href="https://colab.research.google.com/github/vinhngx/DeepLearningExamples/blob/vinhn_unet_industrial_demo/TensorFlow/Segmentation/UNet_Industrial/notebooks/TensorFlow_UNet_Industrial_Colab_train_and_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Copyright 2019 NVIDIA Corporation. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

<img src="http://developer.download.nvidia.com/compute/machine-learning/frameworks/nvidia_logo.png" style="width: 90px; float: right;">

# UNet Industrial Training and Inference Demo

## Overview


This U-Net model is adapted from the original version of the [U-Net model](https://arxiv.org/abs/1505.04597) which is
a convolutional auto-encoder for 2D image segmentation. U-Net was first introduced by
Olaf Ronneberger, Philip Fischer, and Thomas Brox in the paper:
[U-Net: Convolutional Networks for Biomedical Image Segmentation](https://arxiv.org/abs/1505.04597).

This work proposes a modified version of U-Net, called `TinyUNet` which performs efficiently and with very high accuracy
on the industrial anomaly dataset [DAGM2007](https://resources.mpi-inf.mpg.de/conference/dagm/2007/prizes.html).
*TinyUNet*, like the original *U-Net* is composed of two parts:
- an encoding sub-network (left-side)
- a decoding sub-network (right-side).

It repeatedly applies 3 downsampling blocks composed of two 2D convolutions followed by a 2D max pooling
layer in the encoding sub-network. In the decoding sub-network, 3 upsampling blocks are composed of a upsample2D
layer followed by a 2D convolution, a concatenation operation with the residual connection and two 2D convolutions.

`TinyUNet` has been introduced to reduce the model capacity which was leading to a high degree of over-fitting on a
small dataset like DAGM2007. The complete architecture is presented in the figure below:

![UnetModel](https://github.com/vinhngx/DeepLearningExamples/blob/vinhn_unet_industrial_demo/TensorFlow/Segmentation/UNet_Industrial/images/unet.png?raw=1)



### Learning objectives

This notebook demonstrates the steps for training a UNet model. We then employ the trained model to make inference on new images.

## Content
1. [Requirements](#1)
1. [Data download and preprocessing](#2)
1. [Training](#3)
1. [Testing trained model](#4)


<a id="1"></a>
## 1. Requirements


### 1.1 Docker container
The most convenient way to make use of the NVIDIA Mask R-CNN model is via a docker container, which provides a self-contained, isolated and re-producible environment for all experiments. Refer to the [Quick Start Guide section](https://github.com/NVIDIA/DeepLearningExamples/tree/master/PyTorch/Segmentation/MaskRCNN#requirements) of the Readme documentation for a comprehensive guide. We briefly summarize the steps here.

First, clone the repository:

```
git clone https://github.com/NVIDIA/DeepLearningExamples.git
cd DeepLearningExamples/PyTorch/Segmentation/MaskRCNN
```

Next, build the NVIDIA Mask R-CNN container:

```
cd pytorch
docker build --rm -t nvidia_joc_maskrcnn_pt .
```

Then launch the container with:

```
PATH_TO_COCO='/path/to/coco-2014'
MOUNT_LOCATION='/datasets/data'
NAME='nvidia_maskrcnn'

docker run --it --runtime=nvidia -p 8888:8888 -v $PATH_TO_COCO:/$MOUNT_LOCATION --rm --name=$NAME --shm-size=1g --ulimit memlock=-1 --ulimit stack=67108864 --ipc=host nvidia_joc_maskrcnn_pt
```
where `/path/to/coco-2014` is the path on the host machine where the data was/is to be downloaded. More on data set preparation in the next section.

Within the docker interactive bash session, start Jupyter with

```
jupyter notebook --ip 0.0.0.0 --port 8888
```

Then open the Jupyter GUI interface on your host machine at http://localhost:8888. Within the container, this notebook itself is located at `/workspace/object_detection/demo`.

### 1.2 Hardware
This notebook can be executed on any CUDA-enabled NVIDIA GPU, although for efficient mixed precision training, a [Tensor Core NVIDIA GPU](https://www.nvidia.com/en-us/data-center/tensorcore/) is desired (Volta, Turing or newer architectures). 

In [0]:
!nvidia-smi

<a id="2"></a>
## 2. Data download and preprocessing

This notebook demonstrates training and validation of the Mask R-CNN model on the [COCO 2014 dataset](http://cocodataset.org/#download). If not already available locally, the following [script](https://github.com/NVIDIA/DeepLearningExamples/blob/master/PyTorch/Segmentation/MaskRCNN/download_dataset.sh) in the repository provides a convenient way to download and extract all the necessary data in one go. Be mindful of the size of the raw data (~20GB). The script makes use of `wget` and will automatically resume if disrupted. Once downloaded, the script invokes `dtrx` to extract the data.

In [0]:
! wget https://raw.githubusercontent.com/NVIDIA/DeepLearningExamples/master/PyTorch/Segmentation/MaskRCNN/download_dataset.sh -P /workspace/object_detection/
! bash /workspace/object_detection/download_dataset.sh /datasets/data

Within the docker container, the final data directory should look like:

```
/datasets/data
  annotations/
    instances_train2014.json
    instances_val2014.json
  train2014/
    COCO_train2014_*.jpg
  val2014/
    COCO_val2014_*.jpg
```

<a id="3"></a>
## 3. Training

The shell script [train.sh](https://github.com/NVIDIA/DeepLearningExamples/blob/master/PyTorch/Segmentation/MaskRCNN/pytorch/scripts/train.sh) provides a convenient interface to launch training tasks. 
By default, invoking [train.sh](https://github.com/NVIDIA/DeepLearningExamples/blob/master/PyTorch/Segmentation/MaskRCNN/pytorch/scripts/train.sh) will make use of 8 GPUs, saves checkpoints every 2500 iterations and uses mixed precision training.

```
cd /workspace/object_detection/
bash scripts/train.sh
```

Note that, within [train.sh](https://github.com/NVIDIA/DeepLearningExamples/blob/master/PyTorch/Segmentation/MaskRCNN/pytorch/scripts/train.sh), it invokes the following Python command:

```python -m torch.distributed.launch --nproc_per_node=8 tools/train_net.py --config-file "configs/e2e_mask_rcnn_R_50_FPN_1x.yaml" DTYPE "float16"```

which launches pytorch distributed training with 8 GPUs, using the train script in [tools/train_net.py](https://github.com/NVIDIA/DeepLearningExamples/blob/master/PyTorch/Segmentation/MaskRCNN/pytorch/tools/train_net.py). Various sample training configurations are available within the [configs](https://github.com/NVIDIA/DeepLearningExamples/tree/master/PyTorch/Segmentation/MaskRCNN/pytorch/configs) directory, for example, a [configuration file](https://github.com/NVIDIA/DeepLearningExamples/blob/master/PyTorch/Segmentation/MaskRCNN/pytorch/configs/e2e_mask_rcnn_R_50_FPN_1x_1GPU.yaml) for training using 1 GPU. 

### 3.1 Training with 1 GPU
We will now take a closer look at training a Mask-RCNN model using 1 GPU, using the below custom config script.

#### Training with full precision

In [0]:
%run -m torch.distributed.launch -- --nproc_per_node=8 ../tools/train_net.py --config-file "/workspace/object_detection/configs/custom_config_8_GPUs.yml" DTYPE "float32" OUTPUT_DIR ./results/8GPU-FP32/

If the Jupyter graphical interface does not update the training progress on the fly, you can observe the information being printed in the shell window from where you launched Jupyter. 

#### Training with mixed-precision
We now launch the training process using mixed precision. Observe the information being printed in the shell window from where you launched Jupyter. 

In [0]:
%run -m torch.distributed.launch -- --nproc_per_node=8 ../tools/train_net.py --config-file "/workspace/object_detection/configs/custom_config_8_GPUs.yml" DTYPE "float16" OUTPUT_DIR ./results/8GPU-FP16/

<a id="4"></a>
## 4. Testing trained model

After model training has completed, we can test the trained model against the COCO-2014 validation set. First, we create a new configuration file for the test. Note: you must point the model `WEIGHT` parameter to a final model checkpoint, e.g. `./results/8GPU-FP32/model_final.pth`.

In [0]:
%run ../tools/test_net.py \
    --config-file /workspace/object_detection/configs/test_custom_config.yml \
    DTYPE "float16" \
    DATASETS.TEST "(\"coco_2014_minival\",)" \
    OUTPUT_DIR ./results/8GPU-FP16/evaluation \
    TEST.IMS_PER_BATCH 1

### Testing on new images

We will now launch an interactive testing, where you can load new test images. First, we load some required libraries and define some helper functions to load images.

# Conclusion

In this notebook, we have walked through the complete process of preparing the container and data required for training Mask-RCNN models. We have also investigated various training options, trained and tested Mask-RCNN models with various configurations.

## What's next
Now it's time to try the MaskR-CNN on your own data. Observe the performance impact of mixed precision training while comparing the final accuracy of the models trained with FP32 and mixed precision.
